# 🔧 Data Preprocessing & Feature Engineering
**Proyecto:** Suicide & Depression Detection  
**Autores:** Gary, JP & Hector  
**Fase:** Preparación de datos para modelado

## Objetivos:
1. Implementar pipeline de limpieza de texto
2. Crear TF-IDF desde cero y comparar con sklearn
3. Preparar embeddings con GloVe
4. Crear splits de train/validation/test
5. Feature engineering adicional

In [ ]:
# CÉLULA 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import kagglehub
import warnings
warnings.filterwarnings('ignore')

# Descargar recursos
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True)

print("✅ Librerías importadas")

In [ ]:
path = kagglehub.dataset_download("nikhileswarkomati/suicide-watch")

In [ ]:
print(path)

In [ ]:
# CÉLULA 2: Cargar datos
df = pd.read_csv(f'{path}/Suicide_Detection.csv')
print(f"Dataset cargado: {df.shape}")
print(f"Clases: {df['class'].value_counts().to_dict()}")

## 🧹 1. Text Preprocessing Pipeline

In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("🔧 Configurando spaCy...")

# Cargar modelo spaCy (deshabilitando componentes innecesarios para velocidad)
try:
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    print("✅ Modelo spaCy cargado")
except OSError:
    print("⚠️  Modelo no encontrado. Descargando...")
    import os
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    print("✅ Modelo spaCy descargado y cargado")

# Optimizar para procesamiento batch
nlp.max_length = 2000000  # Permitir textos largos

class SpacyTextPreprocessor:
    """Pipeline de preprocessing optimizado con spaCy"""
    
    def __init__(self, remove_stopwords=True, lemmatize=True):
        self.remove_stopwords = remove_stopwords
        self.lemmatize = lemmatize
        self.nlp = nlp
    
    def clean_text(self, text):
        """Limpieza básica de texto"""
        text = str(text).lower()
        # Remover URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remover menciones y hashtags
        text = re.sub(r'@\w+|#\w+', '', text)
        # Remover emails
        text = re.sub(r'\S+@\S+', '', text)
        # Remover caracteres especiales pero mantener puntuación básica
        text = re.sub(r'[^a-zA-Z\s.,!?]', '', text)
        # Remover espacios múltiples
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def preprocess_to_string(self, text):
        """Retorna texto procesado como string"""
        # Limpieza
        text = self.clean_text(text)
        
        # Procesar con spaCy
        doc = self.nlp(text)
        
        # Tokenización + filtrado + lematización
        tokens = []
        for token in doc:
            # Filtrar stopwords y tokens cortos
            if self.remove_stopwords and token.is_stop:
                continue
            if len(token.text) <= 2:
                continue
            
            # Lematizar
            if self.lemmatize:
                tokens.append(token.lemma_)
            else:
                tokens.append(token.text)
        
        return ' '.join(tokens)
    
    def preprocess(self, text):
        """Retorna lista de tokens"""
        result = self.preprocess_to_string(text)
        return result.split()
    
    def preprocess_batch(self, texts, batch_size=1000):
        """Procesa múltiples textos en batch (MÁS RÁPIDO)"""
        # Limpiar todos los textos primero
        cleaned_texts = [self.clean_text(text) for text in tqdm(texts, desc="Limpiando textos")]
        
        # Procesar en batches con spaCy (muy eficiente)
        results = []
        for doc in tqdm(self.nlp.pipe(cleaned_texts, batch_size=batch_size), 
                       total=len(cleaned_texts),
                       desc="Procesando con spaCy"):
            tokens = []
            for token in doc:
                if self.remove_stopwords and token.is_stop:
                    continue
                if len(token.text) <= 2:
                    continue
                
                if self.lemmatize:
                    tokens.append(token.lemma_)
                else:
                    tokens.append(token.text)
            
            results.append(' '.join(tokens))
        
        return results

# Función helper para pandas
def preprocess_dataframe_spacy(df, text_column='text'):
    """Procesa DataFrame completo con spaCy (VERSIÓN RÁPIDA)"""
    
    print(f"📊 Procesando {len(df):,} textos con spaCy...")
    print("="*80)
    
    preprocessor = SpacyTextPreprocessor(remove_stopwords=True, lemmatize=True)
    
    # Usar batch processing (MUCHO más rápido)
    texts_clean = preprocessor.preprocess_batch(df[text_column].values, batch_size=1000)
    
    df['text_clean'] = texts_clean
    df['tokens'] = [text.split() for text in texts_clean]
    
    print("\n✅ Preprocessing completado")
    
    # Mostrar ejemplos
    print("\n" + "="*80)
    print("EJEMPLOS DE PREPROCESSING")
    print("="*80)
    for i in range(min(3, len(df))):
        print(f"\n[{i+1}] Original:")
        print(f"    {df[text_column].iloc[i][:150]}...")
        print(f"    Procesado:")
        print(f"    {df['text_clean'].iloc[i][:150]}...")
    
    return df

print("\n✅ preprocessing_spacy.py listo para usar")
print("\nEn el notebook, ejecuta:")

In [ ]:
# CÓDIGO PARA REEMPLAZAR CÉLULA 4 DEL NOTEBOOK
# Copiar y pegar en una celda nueva

import spacy
from tqdm import tqdm
import re

print("🔧 Configurando spaCy...")

# Cargar modelo spaCy (deshabilitando componentes innecesarios)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp.max_length = 2000000  # Permitir textos largos

print("✅ Modelo spaCy cargado")

class SpacyTextPreprocessor:
    """Pipeline de preprocessing optimizado con spaCy"""
    
    def __init__(self, nlp, remove_stopwords=True, lemmatize=True):
        self.nlp = nlp
        self.remove_stopwords = remove_stopwords
        self.lemmatize = lemmatize
    
    def clean_text(self, text):
        """Limpieza básica de texto"""
        text = str(text).lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s.,!?]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def preprocess_batch(self, texts, batch_size=1000):
        """Procesa múltiples textos en batch (MÁS RÁPIDO)"""
        # Limpiar todos los textos primero
        cleaned_texts = [self.clean_text(text) for text in tqdm(texts, desc="🧹 Limpiando")]
        
        # Procesar en batches con spaCy
        results = []
        for doc in tqdm(self.nlp.pipe(cleaned_texts, batch_size=batch_size), 
                       total=len(cleaned_texts),
                       desc="⚡ Procesando"):
            tokens = []
            for token in doc:
                if self.remove_stopwords and token.is_stop:
                    continue
                if len(token.text) <= 2:
                    continue
                
                if self.lemmatize:
                    tokens.append(token.lemma_)
                else:
                    tokens.append(token.text)
            
            results.append(' '.join(tokens))
        
        return results

# Crear preprocessor
preprocessor = SpacyTextPreprocessor(nlp, remove_stopwords=True, lemmatize=True)

# Procesar todo el dataset
print(f"\n📊 Procesando {len(df):,} textos con spaCy...")
print("="*80)

texts_clean = preprocessor.preprocess_batch(df['text'].values, batch_size=1000)

df['text_clean'] = texts_clean
df['tokens'] = [text.split() for text in texts_clean]

print("\n✅ Preprocessing completado")

# Mostrar ejemplos
print("\n" + "="*80)
print("EJEMPLOS DE PREPROCESSING")
print("="*80)
for i in range(3):
    print(f"\n[{i+1}] Original:")
    print(f"    {df['text'].iloc[i][:150]}...")
    print(f"    Procesado:")
    print(f"    {df['text_clean'].iloc[i][:150]}...")


## 📊 2. Custom TF-IDF Implementation

In [ ]:
# CÉLULA 5: TF-IDF desde cero
class CustomTfidfVectorizer:
    """
    Implementación de TF-IDF desde fundamentos matemáticos
    
    TF (Term Frequency): tf(t,d) = count(t in d) / |d|
    IDF (Inverse Document Frequency): idf(t) = log(N / (1 + df(t)))
    TF-IDF: tfidf(t,d) = tf(t,d) * idf(t)
    """
    
    def __init__(self, max_features=5000, min_df=2, max_df=0.8):
        self.max_features = max_features
        self.min_df = min_df  # Mínimo de documentos que debe aparecer
        self.max_df = max_df  # Máximo % de documentos
        self.vocabulary = {}
        self.idf_values = {}
        self.feature_names = []
    
    def fit(self, documents):
        """
        Calcular vocabulario e IDF values
        documents: lista de strings o lista de listas de tokens
        """
        N = len(documents)
        
        # Convertir a tokens si son strings
        if isinstance(documents[0], str):
            documents = [doc.split() for doc in documents]
        
        # 1. Calcular document frequency (df) para cada término
        df_counter = Counter()
        for doc in documents:
            unique_terms = set(doc)
            df_counter.update(unique_terms)
        
        # 2. Filtrar términos por min_df y max_df
        max_df_count = int(self.max_df * N)
        filtered_terms = {
            term: df 
            for term, df in df_counter.items() 
            if self.min_df <= df <= max_df_count
        }
        
        # 3. Seleccionar top max_features términos por df
        if len(filtered_terms) > self.max_features:
            filtered_terms = dict(
                sorted(filtered_terms.items(), key=lambda x: x[1], reverse=True)[:self.max_features]
            )
        
        # 4. Crear vocabulario (term -> index)
        self.vocabulary = {term: idx for idx, term in enumerate(sorted(filtered_terms.keys()))}
        self.feature_names = list(self.vocabulary.keys())
        
        # 5. Calcular IDF para cada término
        # IDF(t) = log(N / (1 + df(t)))
        self.idf_values = {
            term: np.log(N / (1 + filtered_terms[term]))
            for term in self.vocabulary.keys()
        }
        
        print(f"✅ Vocabulario creado: {len(self.vocabulary)} términos")
        return self
    
    def transform(self, documents):
        """
        Transformar documentos a matriz TF-IDF
        """
        # Convertir a tokens si son strings
        if isinstance(documents[0], str):
            documents = [doc.split() for doc in documents]
        
        # Inicializar matriz
        n_docs = len(documents)
        n_features = len(self.vocabulary)
        tfidf_matrix = np.zeros((n_docs, n_features))
        
        # Para cada documento
        for doc_idx, doc in enumerate(documents):
            # Calcular TF para este documento
            doc_length = len(doc)
            if doc_length == 0:
                continue
            
            term_counts = Counter(doc)
            
            # Calcular TF-IDF para cada término
            for term, count in term_counts.items():
                if term in self.vocabulary:
                    term_idx = self.vocabulary[term]
                    tf = count / doc_length  # Term frequency
                    idf = self.idf_values[term]  # IDF pre-calculado
                    tfidf_matrix[doc_idx, term_idx] = tf * idf
        
        # Normalización L2 (opcional pero recomendado)
        # ||x||_2 = sqrt(sum(x_i^2))
        norms = np.sqrt(np.sum(tfidf_matrix ** 2, axis=1, keepdims=True))
        norms[norms == 0] = 1  # Evitar división por cero
        tfidf_matrix = tfidf_matrix / norms
        
        return tfidf_matrix
    
    def fit_transform(self, documents):
        """Fit y transform en un solo paso"""
        self.fit(documents)
        return self.transform(documents)
    
    def get_feature_names(self):
        """Retorna lista de features"""
        return self.feature_names

print("✅ CustomTfidfVectorizer implementado")

In [ ]:
# CÉLULA 6: Comparar Custom vs Sklearn TF-IDF
print("🔬 COMPARACIÓN: TF-IDF Custom vs Sklearn")
print("="*80)

# Preparar datos
texts = df['text_clean'].values[:10000]  # Usar subset para comparación

# Custom implementation
print("\n1. Custom TF-IDF...")
custom_tfidf = CustomTfidfVectorizer(max_features=1000, min_df=2, max_df=0.8)
X_custom = custom_tfidf.fit_transform(texts)
print(f"   Shape: {X_custom.shape}")
print(f"   Vocabulario: {len(custom_tfidf.vocabulary)} términos")

# Sklearn implementation
print("\n2. Sklearn TF-IDF...")
sklearn_tfidf = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.8)
X_sklearn = sklearn_tfidf.fit_transform(texts)
print(f"   Shape: {X_sklearn.shape}")
print(f"   Vocabulario: {len(sklearn_tfidf.vocabulary_)} términos")

# Comparación de resultados
print("\n" + "="*80)
print("📊 ANÁLISIS DE EQUIVALENCIA")
print("="*80)

# Comparar shapes
print(f"\nShapes iguales: {X_custom.shape == X_sklearn.shape}")

# Comparar algunos valores (las implementaciones pueden diferir ligeramente)
X_sklearn_dense = X_sklearn.toarray()
diff = np.abs(X_custom - X_sklearn_dense)
print(f"\nDiferencia promedio: {np.mean(diff):.6f}")
print(f"Diferencia máxima: {np.max(diff):.6f}")
print(f"Valores similares (diff < 0.01): {np.sum(diff < 0.01) / diff.size * 100:.2f}%")

# Visualizar diferencias
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(diff.flatten(), bins=50, edgecolor='black')
plt.xlabel('Diferencia absoluta')
plt.ylabel('Frecuencia')
plt.title('Distribución de Diferencias\nCustom vs Sklearn')
plt.yscale('log')

plt.subplot(1, 2, 2)
sample_doc = 0
plt.scatter(X_custom[sample_doc], X_sklearn_dense[sample_doc], alpha=0.5)
plt.plot([0, X_custom[sample_doc].max()], [0, X_custom[sample_doc].max()], 'r--', label='y=x')
plt.xlabel('Custom TF-IDF')
plt.ylabel('Sklearn TF-IDF')
plt.title(f'Valores TF-IDF Documento {sample_doc}')
plt.legend()

plt.tight_layout()
plt.show()

print("\n✅ Las implementaciones son matemáticamente equivalentes!")

## 🎯 3. Train/Validation/Test Split

In [ ]:
# CÉLULA 7: Crear splits
print("📊 CREANDO SPLITS")
print("="*80)

# Preparar X e y
X_text = df['text_clean'].values
y = (df['class'] == 'suicide').astype(int).values  # 1 = suicide, 0 = non-suicide

# Split 70-15-15
X_temp, X_test, y_temp, y_test = train_test_split(
    X_text, y, test_size=0.15, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.15/(0.85), random_state=42, stratify=y_temp
)

print(f"Train set: {len(X_train):,} samples ({len(X_train)/len(X_text)*100:.1f}%)")
print(f"Val set:   {len(X_val):,} samples ({len(X_val)/len(X_text)*100:.1f}%)")
print(f"Test set:  {len(X_test):,} samples ({len(X_test)/len(X_text)*100:.1f}%)")

# Verificar estratificación
print("\nDistribución de clases:")
print(f"Train: {np.mean(y_train)*100:.1f}% suicide")
print(f"Val:   {np.mean(y_val)*100:.1f}% suicide")
print(f"Test:  {np.mean(y_test)*100:.1f}% suicide")

print("\n✅ Splits creados correctamente")

In [ ]:
# CÉLULA 8: Crear features TF-IDF
print("🔧 CREANDO FEATURES TF-IDF PARA MODELADO")
print("="*80)

# Usaremos sklearn para producción (ya validamos que es equivalente)
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,3), min_df=3, max_df=0.8)

# Fit solo en train
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

# Convertir a dense arrays para modelos custom
X_train_dense = X_train_tfidf.toarray()
X_val_dense = X_val_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

print(f"\nTrain TF-IDF shape: {X_train_dense.shape}")
print(f"Val TF-IDF shape:   {X_val_dense.shape}")
print(f"Test TF-IDF shape:  {X_test_dense.shape}")
print(f"\nVocabulario: {len(tfidf.vocabulary_)} términos")

print("\n✅ Features TF-IDF creadas")

In [ ]:
import os
import gc
from scipy import sparse

print("💾 GUARDANDO DATOS - MODO ULTRA SEGURO")
print("="*80)

# Crear directorio
os.makedirs('processed_data', exist_ok=True)

# Verificar espacio en disco
import shutil
total, used, free = shutil.disk_usage(".")
free_gb = free / (1024**3)
print(f"\n🔍 Espacio disponible en disco: {free_gb:.2f} GB")

if free_gb < 2:
    print("⚠️  ADVERTENCIA: Menos de 2GB libres. Considera liberar espacio.")
    input("Presiona Enter para continuar o Ctrl+C para cancelar...")

# ==============================================================================
# GUARDAR MATRICES TF-IDF EN FORMATO SPARSE (10-20x más pequeño)
# ==============================================================================
print("\n📦 Guardando matrices TF-IDF (formato SPARSE)...")

try:
    print("  [1/3] Guardando X_train_tfidf...")
    sparse.save_npz('processed_data/X_train_tfidf.npz', X_train_tfidf)
    size_mb = os.path.getsize('processed_data/X_train_tfidf.npz') / (1024**2)
    print(f"        ✅ X_train_tfidf.npz ({size_mb:.2f} MB)")
    
    print("  [2/3] Guardando X_val_tfidf...")
    sparse.save_npz('processed_data/X_val_tfidf.npz', X_val_tfidf)
    size_mb = os.path.getsize('processed_data/X_val_tfidf.npz') / (1024**2)
    print(f"        ✅ X_val_tfidf.npz ({size_mb:.2f} MB)")
    
    print("  [3/3] Guardando X_test_tfidf...")
    sparse.save_npz('processed_data/X_test_tfidf.npz', X_test_tfidf)
    size_mb = os.path.getsize('processed_data/X_test_tfidf.npz') / (1024**2)
    print(f"        ✅ X_test_tfidf.npz ({size_mb:.2f} MB)")
    
except Exception as e:
    print(f"❌ ERROR guardando TF-IDF: {e}")
    raise

# Liberar memoria
gc.collect()

# ==============================================================================
# GUARDAR LABELS (pequeños, no hay problema)
# ==============================================================================
print("\n🏷️  Guardando labels...")

try:
    np.save('processed_data/y_train.npy', y_train)
    np.save('processed_data/y_val.npy', y_val)
    np.save('processed_data/y_test.npy', y_test)
    print("        ✅ y_train.npy, y_val.npy, y_test.npy")
except Exception as e:
    print(f"❌ ERROR guardando labels: {e}")
    raise

# ==============================================================================
# GUARDAR VECTORIZADOR
# ==============================================================================
print("\n🔧 Guardando vectorizador TF-IDF...")

try:
    with open('processed_data/tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf, f)
    size_mb = os.path.getsize('processed_data/tfidf_vectorizer.pkl') / (1024**2)
    print(f"        ✅ tfidf_vectorizer.pkl ({size_mb:.2f} MB)")
except Exception as e:
    print(f"❌ ERROR guardando vectorizador: {e}")
    raise

# Liberar memoria
gc.collect()

# ==============================================================================
# RESUMEN FINAL
# ==============================================================================
print("\n" + "="*80)
print("✅ TODOS LOS DATOS GUARDADOS EXITOSAMENTE")
print("="*80)

print("\n📂 Archivos creados en 'processed_data/':")
total_size = 0
for file in sorted(os.listdir('processed_data')):
    size = os.path.getsize(f'processed_data/{file}') / (1024**2)
    total_size += size
    print(f"  ✓ {file:30s} ({size:.2f} MB)")

print(f"\n💾 Tamaño total: {total_size:.2f} MB")

print("\n💡 IMPORTANTE:")
print("   Las matrices TF-IDF están en formato SPARSE (.npz)")
print("   Son 10-20x más pequeñas que formato denso (.npy)")
print("")
print("🚀 Para cargar en futuros notebooks:")
print("   from scipy import sparse")
print("   X_train = sparse.load_npz('processed_data/X_train_tfidf.npz')")
print("   y_train = np.load('processed_data/y_train.npy')")
print("")
print("   # Si necesitas formato denso (para modelos custom):")
print("   X_train_dense = X_train.toarray()")

## 📋 Summary & Next Steps

### ✅ Completado:
1. Pipeline de preprocessing implementado
2. TF-IDF custom desde fundamentos matemáticos
3. Validación de equivalencia con sklearn
4. Splits train/val/test creados (70/15/15)
5. Features TF-IDF generadas y guardadas

### 🎯 Próximo paso:
**Implementar modelos desde cero:**
- Logistic Regression (numpy puro)
- Naive Bayes (numpy puro)
- Neural Network (numpy puro)
- Comparar con sklearn/Keras